In [1]:
# !wget https://huggingface.co/datasets/mesolitica/embedding-pair-mining/resolve/main/b.cari.com.my-dataset-bge-test.jsonl

In [2]:
# !pip3 install openai

In [3]:
# we use OpenAI Azure, replace it with standard OpenAI

In [4]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [5]:
import json
import openai
from tqdm import tqdm

In [6]:
with open('malay-news-dataset-bge-test.sample.json') as fopen:
    rev_data = json.load(fopen)

In [7]:
vectors = {}
for k, v in tqdm(rev_data.items()):
    try:
        o = openai.Embedding.create(input=k, engine=engine)
        vectors[k] = o['data'][0]['embedding']
    except:
        pass

 24%|██▍       | 4444/18237 [19:01<58:19,  3.94it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 18237/18237 [1:17:59<00:00,  3.90it/s]


In [8]:
no_string = {no: k for no, (k, v) in enumerate(vectors.items())}
string_no = {v: k for k, v in no_string.items()}
no_string[0]

'Sejurus pulih, Subasic yang muncul wira Croatia dalam sepakan penalti menentang Denmark pada pusingan 16 pasukan terakhir, menyelamatkan rembatan Fedor Smolov dalam masa kecederaan.'

In [9]:
with open('openai-malay-news.vectors.json', 'w') as fopen:
    json.dump(vectors, fopen)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [11]:
line = 0
positive, negative = [], []
with open('malay-news-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        v_query = vectors.get(query)
        if v_query is None:
            continue
        v_query = np.array(v_query).reshape((1, -1))
        for s in l['pos']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            positive.append(cosine_similarity(v_query, v_s)[0, 0])
        
        for s in l['neg']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            negative.append(1 - cosine_similarity(v_query, v_s)[0, 0])
            
        line += 1
        if line >= 1000:
            break

999it [00:07, 132.23it/s]


In [13]:
np.mean(positive), np.mean(negative)

(0.8448714707337686, 0.2741472719191583)

In [10]:
np_vectors = np.array(list(vectors.values()))

In [14]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}
total = 0
with open('malay-news-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        query_no = string_no.get(query)
        if query_no is None:
            continue
        for s in l['pos']:
            s = s.strip()
            v_s = vectors.get(s)
            s_no = string_no.get(s)
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            argsort = np.argsort(cosine_similarity(v_s, np_vectors)[0])[::-1]
            for k in tops.keys():
                if s_no in argsort[:k]:
                    k_ = k + 1
                else:
                    k_ = k
                if query_no in argsort[:k_]:
                    tops[k] += 1
            total += 1

19613it [20:43, 15.77it/s] 


In [16]:
tops, total

({1: 1179, 3: 2510, 5: 3184, 10: 4042}, 8501)

In [17]:
for k, v in tops.items():
    print(k, v / total)

1 0.1386895659334196
3 0.2952593812492648
5 0.3745441712739678
10 0.4754734737089754
